**Name: Escalicas Peter Allen**

**Section: BSCPE32S3**

**Date Of Submission: April 8, 2024**

Explain your datasets and the problem being addressed. <br>

My dataset that I used for my Classification problem is Blood Transfusion Service Center. The problem that is being addressed here, is why some donors stop donating and develop strategies to retain existing donors and encourage more blood donor donations.

In [44]:
pip install ucimlrepo

In [45]:
#Importing libraries1
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from keras.models  import Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam, SGD, RMSprop

In [46]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
blood_transfusion_service_center = fetch_ucirepo(id=176)

# data (as pandas dataframes)
X = blood_transfusion_service_center.data.features
y = blood_transfusion_service_center.data.targets

# metadata
print(blood_transfusion_service_center.metadata)

# variable information
print(blood_transfusion_service_center.variables)


{'uci_id': 176, 'name': 'Blood Transfusion Service Center', 'repository_url': 'https://archive.ics.uci.edu/dataset/176/blood+transfusion+service+center', 'data_url': 'https://archive.ics.uci.edu/static/public/176/data.csv', 'abstract': 'Data taken from the Blood Transfusion Service Center in Hsin-Chu City in Taiwan -- this is a classification problem. ', 'area': 'Business', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 748, 'num_features': 4, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Donated_Blood'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2008, 'last_updated': 'Sat Mar 16 2024', 'dataset_doi': '10.24432/C5GS39', 'creators': ['I-Cheng Yeh'], 'intro_paper': {'title': 'Knowledge discovery on RFM model using Bernoulli sequence', 'authors': 'I. Yeh, K. Yang, Tao-Ming Ting', 'published_in': 'Expert systems with applications', 'year': 2009, 'url': 'https://www.semantics

# Preproccessing

In [14]:
#Checking of datatypes
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Recency    748 non-null    int64
 1   Frequency  748 non-null    int64
 2   Monetary   748 non-null    int64
 3   Time       748 non-null    int64
dtypes: int64(4)
memory usage: 23.5 KB


In [15]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   Donated_Blood  748 non-null    int64
dtypes: int64(1)
memory usage: 6.0 KB


In [16]:
#finding of Null Values
X.isnull().sum()

Recency      0
Frequency    0
Monetary     0
Time         0
dtype: int64

Observation:

I have observed from this dataset the X and y sets have no null values. By using the code isnull()sum() we can see the summary if there are Null values in the columns in the Datasets.

In [18]:
# Checking if the dataset is loading
X.head(20)

,Recency,Frequency,Monetary,Time
0,2,50,12500,98
1,0,13,3250,28
2,1,16,4000,35
3,2,20,5000,45
4,1,24,6000,77
5,4,4,1000,4
6,2,7,1750,14
7,1,12,3000,35
8,2,9,2250,22
9,5,46,11500,98


In [19]:
#Describing the Dataset
X.describe()

,Recency,Frequency,Monetary,Time
count,748.000000,748.000000,748.000000,748.000000
mean,9.506684,5.514706,1378.676471,34.282086
std,8.095396,5.839307,1459.826781,24.376714
min,0.000000,1.000000,250.000000,2.000000
25%,2.750000,2.000000,500.000000,16.000000
50%,7.000000,4.000000,1000.000000,28.000000
75%,14.000000,7.000000,1750.000000,50.000000
max,74.000000,50.000000,12500.000000,98.000000


In [20]:
X.shape

(748, 4)

# Creating the Base Model

In [54]:
#Splitting the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [55]:
#Importing the Kfold and Assigning the number of folds
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=5)

#Storing of  Base model Accuracy
acc_basemod = []

In [56]:
for train_index, test_index in folds.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = Sequential([
        Dense(5, input_shape=(4,), activation="relu"),
        Dense(1, activation="sigmoid")
    ])

    model.compile(optimizer = 'RMSprop', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs = 25, batch_size = 32, verbose = 0),
    accuracy = model.evaluate(X_test, y_test, verbose = 0)[1]  # To access the Accuracy value sa accuracy

    #Transferring the values to acc
    acc_basemod.append(accuracy)

print('per fold:', acc_basemod)
print('Base Model Average accuracy:', np.mean(acc_basemod))


per fold: [0.5066666603088379, 0.7733333110809326, 0.46666666865348816, 0.6778523325920105, 0.7651006579399109]
Base Model Average accuracy: 0.637923926115036


Observation:

From what I have observed from my base model since I have only a few hidden layer my accuracy percentage is expected to be quite low.

# Creating the Improved Model


In [57]:
#Importing libraries1
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from keras.models  import Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam, SGD, RMSprop

In [58]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
blood_transfusion_service_center = fetch_ucirepo(id=176)

# data (as pandas dataframes)
X = blood_transfusion_service_center.data.features
y = blood_transfusion_service_center.data.targets

# metadata
print(blood_transfusion_service_center.metadata)

# variable information
print(blood_transfusion_service_center.variables)


{'uci_id': 176, 'name': 'Blood Transfusion Service Center', 'repository_url': 'https://archive.ics.uci.edu/dataset/176/blood+transfusion+service+center', 'data_url': 'https://archive.ics.uci.edu/static/public/176/data.csv', 'abstract': 'Data taken from the Blood Transfusion Service Center in Hsin-Chu City in Taiwan -- this is a classification problem. ', 'area': 'Business', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 748, 'num_features': 4, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Donated_Blood'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2008, 'last_updated': 'Sat Mar 16 2024', 'dataset_doi': '10.24432/C5GS39', 'creators': ['I-Cheng Yeh'], 'intro_paper': {'title': 'Knowledge discovery on RFM model using Bernoulli sequence', 'authors': 'I. Yeh, K. Yang, Tao-Ming Ting', 'published_in': 'Expert systems with applications', 'year': 2009, 'url': 'https://www.semantics

In [59]:
#Splitting the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [60]:
#Importing the Kfold and Assigning the number of folds
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=5)

#Storing of  Base model Accuracy
acc_improvmod = []

In [61]:
for train_index, test_index in folds.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = Sequential([
        Dense(128, input_shape=(4,), activation="relu"),
        Dropout(0.2),
        Dense(63, activation="relu"),
        Dropout(0.2),
        Dense(16, activation="relu"),
        Dropout(0.2),
        Dense(1, activation="sigmoid")
    ])

    model.compile(optimizer = 'ADAM', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs = 25, batch_size = 32, verbose = 0),
    accuracy = model.evaluate(X_test, y_test, verbose = 0)[1]  # To access the Accuracy value sa accuracy

    #Transferring the values to acc
    acc_improvmod.append(accuracy)

print('per fold:', acc_improvmod)
print('Average accuracy:', np.mean(acc_improvmod))

per fold: [0.7599999904632568, 0.7599999904632568, 0.7599999904632568, 0.7651006579399109, 0.7651006579399109]
Average accuracy: 0.7620402574539185


# Conclusion for Classification

In this activity, I have learned and implemented the use of the Kfolding cross validation for this activity. At first I had a hard time finding the most optimal dataset in UCI, then I did some of the Preprocessing of the dataset, proceeded on doing the model itself. From what I have observed from the model is that it is like the previous HOA's but this time, the model rund 5 times in my case since I configured the N_splits to 5 that's why the model ran for 5 times. In my improved base model, it has increased from 63% to 76% when I added more hidden layers in my neural network.



---



# Regression Part

Explain your datasets and the problem being addressed.<br>

My dataset is all about examining the different blood types of alchoholics, due to their excessive drinking habits it can result to a liver problem. In this dataset contains blood tests known to be sensitive to liver issues.

In [1]:
#Importing the Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

from keras.models  import Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam, SGD, RMSprop

In [31]:
#Importing the dataset in UCI
from ucimlrepo import fetch_ucirepo

# fetch dataset
liver_disorders = fetch_ucirepo(id=60)

# data (as pandas dataframes)
X = liver_disorders.data.features
y = liver_disorders.data.targets

# metadata
print(liver_disorders.metadata)

# variable information
print(liver_disorders.variables)


{'uci_id': 60, 'name': 'Liver Disorders', 'repository_url': 'https://archive.ics.uci.edu/dataset/60/liver+disorders', 'data_url': 'https://archive.ics.uci.edu/static/public/60/data.csv', 'abstract': 'BUPA Medical Research Ltd. database donated by Richard S. Forsyth', 'area': 'Health and Medicine', 'tasks': ['Regression'], 'characteristics': ['Multivariate'], 'num_instances': 345, 'num_features': 5, 'feature_types': ['Categorical', 'Integer', 'Real'], 'demographics': [], 'target_col': ['drinks'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2016, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C54G67', 'creators': [], 'intro_paper': None, 'additional_info': {'summary': 'The first 5 variables are all blood tests which are thought to be sensitive to liver disorders that might arise from excessive alcohol consumption. Each line in the dataset constitutes the record of a single male individual.\n\nImportant note: The 7

# Preprocessing

In [32]:
#Checking info for X
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345 entries, 0 to 344
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   mcv      345 non-null    int64
 1   alkphos  345 non-null    int64
 2   sgpt     345 non-null    int64
 3   sgot     345 non-null    int64
 4   gammagt  345 non-null    int64
dtypes: int64(5)
memory usage: 13.6 KB


In [33]:
#Countinig the total null
X.isnull().sum()

mcv        0
alkphos    0
sgpt       0
sgot       0
gammagt    0
dtype: int64

Observation:
For our X data, it seems that there are no missing values in any column.

In [34]:
# Checking info for y
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345 entries, 0 to 344
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   drinks  345 non-null    float64
dtypes: float64(1)
memory usage: 2.8 KB


In [35]:
y.isnull().sum()

drinks    0
dtype: int64

In [36]:
#Cheking the Inputs for our model using the .shape code
X.shape

(345, 5)

In [37]:
# Split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [38]:
# Define the base model
base_model = Sequential([
      Dense(64, input_shape=(5,), activation='relu'),
      Dense(1, activation = 'linear')
  ])

# Compile the small model
base_model.compile(optimizer='adam', loss='mse', metrics=["mae"])

# Train the small model
bm = base_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Evaluate the small model on training data
train_loss, train_mae = base_model.evaluate(X_train, y_train, verbose=0)

# Evaluate the small model on test data
test_loss, test_mae = base_model.evaluate(X_test, y_test, verbose=0)

print("Train Loss:", train_loss)
print("Train MAE:", train_mae)
print("Test Loss:", test_loss)
print("Test MAE:", test_mae)

Train Loss: 9.567611694335938
Train MAE: 2.3367507457733154
Test Loss: 12.469160079956055
Test MAE: 2.521751642227173


Observation for Base Model:

From what I can observe from the output of my model, is that since I used 64 nodes in the first layer, it produced or it can learn more things about the relationship of my data variables with each other, resulting to a much lower Train and Test loss&MAE.

In [39]:
# Standardize the dataset
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [40]:
small_model = Sequential([
    Dense(15, input_shape=(5,), activation='relu'),
    Dense(10, activation='relu'),
    Dense(1, activation = 'linear')
])

# Compile the small model
small_model.compile(optimizer='adam', loss='mse', metrics=["mae"])

# Train the small model
sm = small_model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, verbose=0)

# Evaluate the small model on training data
train_loss, train_mae = small_model.evaluate(X_train_scaled, y_train, verbose=0)

# Evaluate the small model on test data
test_loss, test_mae = small_model.evaluate(X_test_scaled, y_test, verbose=0)

print("Train Loss:", train_loss)
print("Train MAE:", train_mae)
print("Test Loss:", test_loss)
print("Test MAE:", test_mae)

Train Loss: 12.741066932678223
Train MAE: 2.5723483562469482
Test Loss: 11.98083209991455
Test MAE: 2.623093843460083


Observation for Small model:

From what I can observe from my small model is that comparing the output of my small model and Base model, the base model has a lower percentage due to the face that it has more nodes than the small networks. I think that is one reason why my train and test loss is more bigger compared to my base model output.

In [42]:
# Define the large model
large_model = Sequential([
    Dense(128, input_shape=(5,), activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation = 'linear')
])

# Compile the small model
large_model.compile(optimizer='adam', loss='mse', metrics=["mae"])

# Train the small model
lm = large_model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=0)

# Evaluate the small model on training data
train_loss, train_mae = large_model.evaluate(X_train_scaled, y_train, verbose=0)

# Evaluate the small model on test data
test_loss, test_mae = large_model.evaluate(X_test_scaled, y_test, verbose=0)

print("Train Loss:", train_loss)
print("Train MAE:", train_mae)
print("Test Loss:", test_loss)
print("Test MAE:", test_mae)

Train Loss: 6.735581398010254
Train MAE: 2.0186667442321777
Test Loss: 11.056135177612305
Test MAE: 2.597517251968384


Observation for Large Network:

For this Large model, I have used 4 hidden layers, it starts at a big nodes up until the output layer. From what I have observed on this models performance, It has a lower Train Test loss and MAE, since It has more hidden layers and more nodes. It means having more hidden layer and nodes for this dataset wil result to a lower loss and MAE comapred to using a small network.

Conclusion:

I have learned many things in this Hand on Activity 7.1, mainly what I did differently this time is that I now imported my datasets that I used for Regression and Classification. I have noticed that its much easier and faster to load when we are using our personal computers compared using the computers in the school. For my regression I have learned on how to implement the K folding and how it works. It basically runs the model n times depending on how many fold you have set for it. And it will output the accuracies that you have entered as the Kfold, in my case it has an output of 5 accuracies because i set my folds = 5 hence that is why I have 5 accuracies. For my regression part of the Activity I have used the Dataset Liver disorder from UCI. My code consists of 3 parts, one for the base model, improving the base model using Stardadization of data, lastly the small and large network part. To summarize your model will have a lower loss and test MAE if you have more hidden layers, ofcourse it need to be finely tuned because having too much hidden layers may result to overfitting to the model.